In [1]:
import os
from pprint import pprint
from dotenv import load_dotenv
from langchain_neo4j import Neo4jGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_anthropic import ChatAnthropic
from src.grag import create_text2cypher_retriever_tool
from src.grag import text2cypher_eval_workflow
from ragas import EvaluationDataset

load_dotenv(".env")

True

In [2]:
URI = os.environ["DATABASE_HOST"]
DATABASE = os.environ["DATABASE_SMALL"]
USERNAME = os.environ["DATABASE_USERNAME"]
PASSWORD = os.environ["DATABASE_PASSWORD"]

neo4j_config = {
    "DATABASE_NAME": DATABASE,
    "ARTICLE_VECTOR_INDEX_NAME": os.environ["ARTICLE_VECTOR_INDEX_NAME"],
    "ARTICLE_FULLTEXT_INDEX_NAME": os.environ["ARTICLE_FULLTEXT_INDEX_NAME"],
    "DEFINITION_VECTOR_INDEX_NAME": os.environ["DEFINITION_VECTOR_INDEX_NAME"],
    "DEFINITION_FULLTEXT_INDEX_NAME": os.environ["DEFINITION_FULLTEXT_INDEX_NAME"],
}

# Ngga jadi pakai ini karena Text2Cypher mintannya harus pakai Neo4jGraph
# neo4j_driver = GraphDatabase.driver(uri=URI, auth=(USERNAME, PASSWORD))

neo4j_graph = Neo4jGraph(
    url=URI,
    username=USERNAME,
    password=PASSWORD,
    database=DATABASE,
    enhanced_schema=True
)

neo4j_driver = neo4j_graph._driver  # Ambil driver nya kaya gini
embedder_model = HuggingFaceEmbeddings(model_name=os.environ["EMBEDDING_MODEL"])

gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
    api_key=os.environ["GOOGLE_API_KEY"]
)

claude_llm = ChatAnthropic(
    model_name="claude-3-5-haiku-20241022",
    temperature=0.0,
    api_key=os.environ["ANTHROPIC_API_KEY"]
)

gpt_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key=os.environ["OPENAI_API_KEY"]
)

In [14]:
evaluation_dataset = EvaluationDataset.from_list([
    {
        "user_input": (
            "Dalam pembangunan infrastruktur telekomunikasi, bagaimana cara perhitungan persentase TKDN untuk belanja modal atau capital expenditure (Capex) yang digunakan?"
        ),
        "reference_contexts": [
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Penilaian Pencapaian Tingkat Komponen Dalam Negeri pada Penyelenggaraan Telekomunikasi, BAB III - TINGKAT KOMPONEN DALAM NEGERI, Pasal 3:\n(1) Persentase TKDN untuk belanja modal (capex) sebagaimana dimaksud dalam Pasal 2 dihitung berdasarkan :\n\ta. perbandingan antara total belanja modal (capex) dikurangi belanja modal (capex) Komponen luar negeri terhadap total belanja modal (capex); atau\n\tb. perbandingan antara total belanja modal (capex) Komponen dalam negeri terhadap total belanja modal (capex);\n(2) Formula TKDN belanja modal (capex) sebagaimana dimaksud pada ayat (1) didasarkan pada rasio sebagai berikut :\n$$\n\\% TKDN (Capex) = \\frac{(Total\\ Capex - Capex\\ Komponen\\ Luar\\ Negeri) \\times 100\\%}{Total\\ Capex}\n$$\natau\n$$\n\\% TKDN (Capex) = \\frac{Capex\\ Komponen\\ Dalam\\ Negeri \\times 100\\%}{Total\\ Capex}\n$$\n(3) Belanja modal (capex) sebagaimana dimaksud pada ayat (1) meliputi biaya pengeluaran untuk :\n\ta. engineering untuk kegiatan perekayasaan pembangunan infrastruktur telekomunikasi;\n\tb. material/perangkat telekomunikasi yang dipergunakan pada pembangunan infrastruktur telekomunikasi;\n\tc. tenaga kerja pada pembangunan infrastruktur telekomunikasi;\n\td. alat kerja yang dipergunakan pada pembangunan infrastruktur telekomunikasi;\n\te. biaya jasa pendukung pada pembangunan infrastruktur telekomunikasi.\n(4) Format dan formula perhitungan TKDN sebagaimana dimaksud pada ayat (2) dan ayat (3) adalah sebagai berikut :\n| No. | Komponen Capex                        | Nilai Capex (Rp) | Capex KDN (Rp) | Capex KLN (Rp) | % TKDN                          |\n|-----|----------------------------------------|------------------|----------------|----------------|--------------------------------|\n|     | (a)                                    | (b)              | (c)            | (d)            | (e) = (c)/(b) atau (b-d)/(b)  |\n| 1.  | Engineering                            | b1               | c1             | d1             | e1 = c1/b1 atau (b1-d1)/b1    |\n| 2.  | Material/Perangkat Telekomunikasi      | b2               | c2             | d2             | e2 = c2/b2 atau (b2-d2)/b2    |\n| 3.  | Tenaga Kerja                           | b3               | c3             | d3             | e3 = c3/b3 atau (b3-d3)/b3    |\n| 4.  | Alat Kerja                             | b4               | c4             | d4             | e4 = c4/b4 atau (b4-d4)/b4    |\n| 5.  | Jasa Pendukung                         | b5               | c5             | d5             | e5 = c5/b5 atau (b5-d5)/b5    |\n|     | **Total**                              | **Σb = b1+...+b5** | **Σc = c1+...+c5** | **Σd = d1+...+d5** | **Σe = Σc/Σb atau (Σb-Σd)/Σb** |",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Penilaian Pencapaian Tingkat Komponen Dalam Negeri pada Penyelenggaraan Telekomunikasi, BAB III - TINGKAT KOMPONEN DALAM NEGERI, Pasal 7:\n(1) Besarnya nilai belanja modal (capex) Komponen dalam negeri yang dihitung dalam rupiah dihitung berdasarkan formula sebagai berikut :\n\tNilai Capex KDN (Rp.) = (nilai perolehan komponen Capex(Rp.)) x % TKDN komponen Capex\n(2) Dalam hal penyelenggara telekomunikasi melakukan pembelanjaan material/perangkat telekomunikasi dengan menggunakan mata uang selain rupiah, maka mata uang dimaksud harus dikonversikan dalam hitungan rupiah berdasarkan rate Bank Indonesia (BI) yang berlaku pada saat pembelanjaan dilakukan.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Penilaian Pencapaian Tingkat Komponen Dalam Negeri pada Penyelenggaraan Telekomunikasi, BAB IV - PENILAIAN SENDIRI DAN VERIFIKASI TINGKAT KOMPONEN DALAM NEGERI, Pasal 8:\n(1) Penyelenggara telekomunikasi wajib menilai sendiri (self assessment) pencapaian TKDN belanja modal (capex) sebagaimana dimaksud dalam Pasal 3 ayat (2) setiap tahun;\n(2) Penilaian sendiri (self assessment) pencapaian TKDN belanja modal (capex) sebagaimana dimaksud pada ayat (1) dilaksanakan berdasarkan data yang dapat dipertanggung jawabkan meliputi :\n\ta. data yang dimiliki sendiri;\n\tb. data yang dimiliki industri barang/jasa (vendor); dan\n\tc. daftar inventarisasi barang/jasa Komponen dalam negeri yang diterbitkan oleh instansi yang berwenang.\n(3) Komponen yang diajukan dalam penilaian sendiri (self assessment) yang tidak berdasarkan data yang dapat dipertanggungjawabkan, dinyatakan sebagai Komponen luar negeri.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Penilaian Pencapaian Tingkat Komponen Dalam Negeri pada Penyelenggaraan Telekomunikasi, BAB V - PELAPORAN, Pasal 10:\n(1) Penyelenggara telekomunikasi wajib melaporkan pencapaian TKDN hasil penilaian sendiri (self assessment) sebagaimana dimaksud dalam Pasal 3 ayat (2) kepada Direktur Jenderal paling lama 3 (tiga) bulan setelah akhir tahun berjalan.\n(2) Laporan pencapaian TKDN sebagaimana dimaksud pada ayat (1) disampaikan sesuai dengan format dengan mengacu pada penjelasan sebagaimana dimaksud dalam Lampiran I dan Lampiran II Peraturan Menteri ini.\n(3) Laporan pencapaian TKDN sebagaimana dimaksud pada ayat (1) harus disertai dengan data pendukung antara lain :\n\ta. dokumen kontrak;\n\tb. kuitansi (invoice);\n\tc. sertifikat negara asal (certificate of origin) ;\n\td. daftar pemasok (vendor);\n\te. sertifikat TKDN.\n(4) Laporan pencapaian TKDN sebagaimana dimaksud pada ayat (1) harus disertai dengan surat pernyataan bermeterai bahwa laporan yang dibuat adalah benar dan akurat serta ditandatangani oleh Direktur Utama atau pejabat yang diberi kewenangan."
        ]
    },
    {
        "user_input": (
            "Apakah Lembaga Penyiaran Asing boleh mendirikan stasiun penyiaran di Indonesia?"
        ),
        "reference_contexts": [
            "Peraturan Menteri Komunikasi dan Informatika Nomor 42/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Memperoleh Izin bagi Lembaga Penyiaran Asing yang Melakukan Kegiatan Peliputan di Indonesia, BAB II - KEGIATAN PELIPUTAN, Pasal 2:\nLembaga Penyiaran Asing dilarang didirikan di Indonesia.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 42/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Memperoleh Izin bagi Lembaga Penyiaran Asing yang Melakukan Kegiatan Peliputan di Indonesia, BAB II - KEGIATAN PELIPUTAN, Pasal 3:\nLembaga Penyiaran Asing hanya dapat menyelenggarakan kegiatan peliputan di Indonesia, yang meliputi :\na. kegiatan, siaran secara tidak tetap ; dan/atau\nb. kegiatan jurnalistik.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 42/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Memperoleh Izin bagi Lembaga Penyiaran Asing yang Melakukan Kegiatan Peliputan di Indonesia, BAB II - KEGIATAN PELIPUTAN, Pasal 4:\n(1) Lembaga Penyiaran Asing yang menyelenggarakan kegiatan siaran secara tidak tetap di Indonesia sebagaimana dimaksud dalam Pasal 3 dapat membawa perangkat pengiriman ke dan penerima siaran dari satelit dan/atau media lainnya.\n(2) Lembaga Penyiaran Asing yang menyelenggarakan kegiatan jurnalistik di Indonesia sebagaimana dimaksud dalam Pasal 3 dapat :\n\ta. menempatkan koresponden untuk melakukan kegiatan jurnalistik; atau\n\tb. membuka kantor penyiaran asing untuk mendukung bidang administratif.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 42/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Memperoleh Izin bagi Lembaga Penyiaran Asing yang Melakukan Kegiatan Peliputan di Indonesia, BAB IV - KANTOR PENYIARAN ASING, Pasal 9:\nDalam membuka kantor penyiaran asing sebagaimana dimaksud dalam Pasal 4 ayat (2) huruf b, Lembaga Penyiaran Asing wajib memenuhi ketentuan sebagai berikut :\na. kantor penyiaran asing tersebut bukan merupakan stasiun penyiaran; dan\nb. kantor penyiaran asing tersebut berlokasi di ibukota negara dan berada pada wilayah yurisdiksi Negara Republik Indonesia."
        ]
    },
    {
        "user_input": (
            "Bagaimana cara mendapatkan rekomendasi untuk menyelenggarakan Diklat Radio Elektronika atau Operator Radio (REOR)?"
        ),
        "reference_contexts": [
            "Peraturan Menteri Komunikasi dan Informatika Nomor 2/PER/M.KOMINFO/3/2011 Tahun 2011 tentang Sertifikasi Radio Elektronika dan Operator Radio, BAB II - PENYELENGGARAAN SERTIFIKASI, Bagian Kedua - Lembaga Pendidikan dan Pelatihan, Pasal 3:\n(1) Pelaksanaan Diklat REOR sebagaimana dimaksud dalam Pasal 2 ayat (3) dilakukan oleh Lembaga Diklat yang telah memenuhi persyaratan sebagai berikut :\n\ta. Lembaga Diklat Pemerintah atau Lembaga Diklat yang berbadan hukum Indonesia; dan\n\tb. mendapatkan rekomendasi penyelenggaraan Diklat dari Direktur Jenderal.\n(2) Untuk mendapatkan rekomendasi sebagaimana dimaksud pada ayat (1) huruf b, Lembaga Diklat mengajukan permohonan kepada Direktur Jenderal dengan dilengkapi persyaratan administrasi dan perlengkapan teknis termasuk sarana dan prasarana sebagai berikut:\n\ta. akta pendirian yang telah disahkan oleh instansi yang berwenang, yang di dalam Anggaran Dasar pendiriannya mencantumkan bidang penyelenggaraan Diklat REOR;\n\tb. Nomor Pokok Wajib Pajak (NPWP);\n\tc. mempunyai instruktur yang berpengalaman sekurang-kurangnya 6 (enam) orang;\n\td. menyediakan seluruh peralatan dan perangkat pendidikan;\n\te. tempat yang tetap untuk menyelenggarakan Diklat; dan\n\tf. mempunyai fasilitas kepustakaan sebagai sarana Diklat.\n(3) Direktur Jenderal melakukan evaluasi persyaratan administrasi dan perlengkapan teknis termasuk sarana dan prasarana sebagaimana dimaksud pada ayat (2) di lokasi Lembaga Diklat REOR.\n(4) Lembaga Diklat REOR sebagaimana dimaksud pada ayat (1) wajib menerapkan kurikulum dan silabus yang ditetapkan oleh Direktur Jenderal.\n(5) Lembaga Diklat REOR sebagaimana dimaksud pada ayat (1) wajib menyampaikan laporan kegiatan penyelenggaraan Diklat REOR per semester kepada Direktur Jenderal.\n(6) Direktur Jenderal membentuk Tim untuk melakukan evaluasi terhadap penerapan kurikulum dan silabus, dan pelaporan kegiatan penyelenggaraan Diklat REOR.\n(7) Tata cara evaluasi terhadap penerapan kurikulum dan silabus, dan pelaporan kegiatan penyelenggaraan Diklat REOR diatur dengan Peraturan Direktur Jenderal.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 2/PER/M.KOMINFO/3/2011 Tahun 2011 tentang Sertifikasi Radio Elektronika dan Operator Radio, BAB IV - PEMBINAAN DAN PENGAWASAN, Pasal 12:\nDirektorat Jenderal melaksanakan fungsi pembinaan dan pengawasan terhadap pelaksanaan Diklat REOR yang diselenggarakan oleh Lembaga Diklat REOR.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 2/PER/M.KOMINFO/3/2011 Tahun 2011 tentang Sertifikasi Radio Elektronika dan Operator Radio, BAB V - SANKSI, Pasal 13:\n(1) Lembaga Diklat REOR yang terbukti tidak menerapkan kurikulum dan silabus sebagaimana dimaksud dalam Pasal 3 ayat (4) atau tidak mengirimkan laporan sebagaimana dimaksud dalam Pasal 3 ayat (5), diberi peringatan tertulis sebanyak 3 (tiga) kali dengan tenggang waktu 30 (tiga puluh) hari kalender untuk setiap peringatan.\n(2) Dalam hal Lembaga Diklat REOR telah diperingatkan 3 (tiga) kali berturut-turut dan tidak ditindaklanjuti, maka Lembaga Diklat dimaksud dilarang menyelenggarakan Diklat REOR.\n(3) Calon pemegang Sertifikat Kewenangan yang tidak mengucapkan sumpah atau janji sebagaimana dimaksud dalam Pasal 10 tidak akan diberikan Sertifikat Kewenangan sebagaimana dimaksud dalam Pasal 2 ayat (2)."
        ]
    },
    {
        "user_input": (
            "Apa perbedaan kewajiban teknis antara penyiaran berlangganan melalui satelit, kabel, dan terestrial?"
        ),
        "reference_contexts": [
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41 Tahun 2012 tentang Penyelenggaraan Penyiaran Lembaga Penyiaran Berlangganan Melalui Satelit, Kabel, dan Terestrial, BAB IV - PENYELENGGARAAN PENYIARAN BERLANGGANAN MELALUI SATELIT, Pasal 14:\nPenyelenggaraan Penyiaran Berlangganan melalui satelit sebagaimana dimaksud dalam Pasal 2 ayat (1) huruf a harus memenuhi ketentuan sebagai berikut:\na. memiliki jangkauan siaran yang dapat diterima di seluruh atau sebagian wilayah Negara Kesatuan Republik Indonesia;\nb. memiliki stasiun pengendali siaran yang berlokasi di Indonesia;\nc. memiliki stasiun pemancar ke satelit yang berlokasi di Indonesia;\nd. dalam hal menggunakan satelit asing harus mempunyai Hak Labuh (landing right) di Indonesia sesuai dengan ketentuan peraturan perundang-undangan; dan\ne. menjamin agar siarannya hanya diterima oleh Pelanggan.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41 Tahun 2012 tentang Penyelenggaraan Penyiaran Lembaga Penyiaran Berlangganan Melalui Satelit, Kabel, dan Terestrial, BAB V - PENYELENGGARAAN PENYIARAN BERLANGGANAN MELALUI KABEL, Pasal 15:\nPenyelenggaraan Penyiaran Berlangganan melalui kabel sebagaimana dimaksud dalam Pasal 2 ayat (1) huruf b harus memenuhi ketentuan sebagai berikut:\na. memiliki jangkauan siaran yang meliputi satu daerah layanan sesuai dengan izin yang diberikan; dan\nb. menjamin agar siarannya hanya diterima oleh Pelanggan.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41 Tahun 2012 tentang Penyelenggaraan Penyiaran Lembaga Penyiaran Berlangganan Melalui Satelit, Kabel, dan Terestrial, BAB V - PENYELENGGARAAN PENYIARAN BERLANGGANAN MELALUI KABEL, Pasal 16:\n(1) Lembaga Penyiaran Berlangganan melalui kabel dapat melakukan pengembangan jangkauan wilayah siaran di luar jangkauan wilayah siaran sebagaimana tercantum dalam izinnya dengan terlebih dahulu melaporkan kepada Menteri.\n(2) Lembaga Penyiaran Berlangganan melalui kabel yang melakukan pengembangan jangkauan wilayah siaran sebagaimana dimaksud pada ayat (1) wajib memenuhi ketentuan peraturan perundang-undangan.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41 Tahun 2012 tentang Penyelenggaraan Penyiaran Lembaga Penyiaran Berlangganan Melalui Satelit, Kabel, dan Terestrial, BAB V - PENYELENGGARAAN PENYIARAN BERLANGGANAN MELALUI KABEL, Pasal 17:\nSetelah menerima laporan sebagaimana dimaksud dalam Pasal 16 ayat (1), Menteri melakukan penambahan jangkauan wilayah siaran dalam Izin Penyelenggaraan Penyiaran.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41 Tahun 2012 tentang Penyelenggaraan Penyiaran Lembaga Penyiaran Berlangganan Melalui Satelit, Kabel, dan Terestrial, BAB VI - PENYELENGGARAAN PENYIARAN BERLANGGANAN MELALUI TERESTRIAL, Pasal 19:\n(1) Penyelenggaraan Penyiaran Berlangganan melalui terestrial sebagaimana dimaksud dalam Pasal 2 ayat (1) huruf c, harus memenuhi ketentuan sebagai berikut:\n\ta. memiliki izin penyelenggaraan penyiaran jasa televisi berlangganan dari Menteri;\n\tb. jangkauan siaran meliputi satu daerah layanan sesuai dengan izin yang diberikan; dan\n\tc. menjamin agar siarannya hanya diterima oleh Pelanggan.\n(2) Izin sebagaimana dimaksud pada ayat (1) huruf a dapat diterbitkan sepanjang Spektrum Frekuensi Radio tersedia.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41 Tahun 2012 tentang Penyelenggaraan Penyiaran Lembaga Penyiaran Berlangganan Melalui Satelit, Kabel, dan Terestrial, BAB VI - PENYELENGGARAAN PENYIARAN BERLANGGANAN MELALUI TERESTRIAL, Pasal 20:\n(1) Jangkauan wilayah siaran Lembaga Penyiaran Berlangganan melalui terestrial sesuai dengan masterplan dan ketersediaan alokasi frekuensi.\n(2) Lembaga Penyiaran Berlangganan melalui terestrial dapat melakukan pengembangan jangkauan wilayah siaran diluar jangkauan wilayah siaran sebagaimana dimaksud pada ayat (1) dengan ketentuan sebagai berikut:\n\ta. tersedianya alokasi frekuensi radio; dan\n\tb. terlebih dahulu wajib mengajukan permohonan kepada Menteri untuk mendapatkan persetujuan."
        ]
    },
    {
        "user_input": (
            "Apa itu Jaringan Dokumentasi dan Informasi Hukum (JDIH) Kemkominfo, dan apa tujuannya?"
        ),
        "reference_contexts": [
            "Peraturan Menteri Komunikasi dan Informatika Nomor 20 Tahun 2013 tentang Jaringan Dokumentasi dan Informasi Hukum Kementerian Komunikasi dan Informatika, BAB I - KETENTUAN UMUM, Pasal 1:\nDalam Peraturan Menteri ini yang dimaksud dengan:\n(1) Jaringan Dokumentasi dan Informasi Hukum Kementerian Komunikasi dan Informatika yang selanjutnya disebut JDIH Kemkominfo adalah wadah pendayagunaan bersama atas dokumen hukum secara tertib, terpadu, dan berkesinambungan, serta merupakan sarana pemberian pelayanan informasi hukum secara lengkap, akurat, mudah, dan tepat di lingkungan Kementerian Komunikasi dan Informatika.\n(2) Dokumen Hukum adalah produk hukum yang berupa peraturan perundang-undangan dan produk hukum selain peraturan perundang-undangan antara lain, putusan pengadilan, yurisprudensi, monografi hukum, artikel majalah hukum, buku hukum, penelitian hukum, pengkajian hukum, naskah akademis, dan rancangan peraturan perundang-undangan.\n(3) Pengelolaan Dokumentasi dan Informasi Hukum adalah kegiatan pengumpulan, pengolahan, penyimpanan, pelestarian, dan pendayagunaan informasi Dokumen Hukum.\n(4) Pusat Jaringan Dokumentasi dan Informasi Hukum Nasional yang selanjutnya disebut Pusat JDIHN adalah Badan Pembinaan Hukum Nasional, Kementerian Hukum dan Hak Asasi Manusia yang bertugas melakukan pembinaan, pengembangan, dan monitoring pada Anggota Jaringan Dokumentasi dan Informasi Hukum Nasional.\n(5) Anggota Jaringan Dokumentasi dan Informasi Hukum Nasional yang selanjutnya disebut Anggota JDIHN adalah kementerian negara, sekretariat lembaga negara, lembaga pemerintahan non kementerian, pemerintah provinsi, pemerintah kabupaten/kota, sekretariat dewan perwakilan rakyat daerah tingkat provinsi dan kabupaten/kota yang tugas dan fungsinya menyelenggarakan kegiatan yang berkaitan dengan Dokumen Hukum, dan perpustakaan hukum pada perguruan tinggi negeri dan perguruan tinggi swasta, serta lembaga lain yang bergerak di bidang pengembangan dokumentasi dan informasi hukum yang ditetapkan oleh menteri hukum dan hak asasi manusia.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 20 Tahun 2013 tentang Jaringan Dokumentasi dan Informasi Hukum Kementerian Komunikasi dan Informatika, BAB I - KETENTUAN UMUM, Pasal 2:\n(1) Peraturan Menteri ini dimaksudkan untuk memberikan kepastian hukum dan kemanfaatan JDIH Kemkominfo.\n(2) Peraturan Menteri ini bertujuan untuk:\n\ta. menjamin terciptanya Pengelolaan Dokumentasi dan Informasi Hukum yang terpadu di lingkungan Kementerian Komunikasi dan Informatika dan terintegrasi dengan Pusat JDIHN dan sesama Anggota JDIHN;\n\tb. menjamin ketersediaan dokumen dan informasi hukum yang lengkap dan akurat, serta dapat diakses secara cepat dan mudah;\n\tc. mengembangkan kerja sama yang efektif dalam rangka penyelenggaraan JDIH Kemkominfo; dan\n\td. meningkatkan kualitas pembangunan hukum di bidang komunikasi dan informatika, serta pelayanan kepada publik sebagai salah satu wujud ketatapemerintahan yang baik, transparan, efektif, efisien, dan bertanggung jawab."
        ]
    },

    # TEXT2CYPHER
    {
        "user_input": "Apa judul dari Peraturan Pemerintah (PP) Nomor 80 Tahun 2019?",
        "reference": "Judul dari Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 adalah Perdagangan Melalui Sistem Elektronik.",
        "reference_contexts": [
            "Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 tentang Perdagangan Melalui Sistem Elektronik"
        ]
    },
    {
        "user_input": "Apa isi Pasal 2 Peraturan Pemerintah (PP) Nomor 80 Tahun 2019?",
        "reference": (
            """
            Isi Pasal 2 Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 tentang Perdagangan Melalui Sistem Elektronik adalah:

            Lingkup pengaturan Perdagangan Melalui Sistem Elektronik meliputi:
            a. pihak yang melakukan PMSE;
            b. persyaratan dalam PMSE;
            c. penyelenggaraan PMSE;
            d. kewajiban Pelaku Usaha;
            e. bukti transaksi PMSE;
            f. Iklan Elektronik;
            g. Penawaran Secara Elektronik, Penerimaan Secara Elektronik, dan Konfirmasi Elektronik;
            h. Kontrak Elektronik;
            i. perlindungan terhadap data pribadi;
            j. pembayaran dalam PMSE;
            k. pengiriman Barang dan Jasa dalam PMSE;
            l. penukaran Barang atau Jasa dan pembatalan pembelian dalam PMSE;
            m. penyelesaian sengketa dalam PMSE; dan
            n. pembinaan dan pengawasan.
            """
        ).strip(),
        "reference_contexts": [
            "Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 tentang Perdagangan Melalui Sistem Elektronik, BAB II - LINGKUP PENGATURAN DAN PRINSIP PERDAGANGAN MELALUI SISTEM ELEKTRONIK, Pasal 2:\nLingkup pengaturan Perdagangan Melalui Sistem Elektronik meliputi:\na. pihak yang melakukan PMSE;\nb. persyaratan dalam PMSE;\nc. penyelenggaraan PMSE;\nd. kewajiban Pelaku Usaha;\ne. bukti transaksi PMSE;\nf. Iklan Elektronik;\ng. Penawaran Secara Elektronik, Penerimaan Secara Elektronik, dan Konfirmasi Elektronik;\nh. Kontrak Elektronik;\ni. perlindungan terhadap data pribadi;\nj. pembayaran dalam PMSE;\nk. pengiriman Barang dan Jasa dalam PMSE;\n1. penukaran Barang atau Jasa dan pembatalan pembelian dalam PMSE;\nm. penyelesaian sengketa dalam PMSE; dan\nn. pembinaan dan pengawasan."
        ]
    },
    {
        "user_input": "Apa isi Pasal 10 Peraturan Menteri Komunikasi dan Informatika (PERMENKOMINFO) Nomor 4 Tahun 2016?",
        "reference": (
            """
            Isi Pasal 10 Peraturan Menteri Komunikasi dan Informatika (PERMENKOMINFO) Nomor 4 Tahun 2016 adalah:
            (1) Penyelenggara Sistem Elektronik strategis dan Penyelenggara Sistem Elektronik tinggi wajib memiliki Sertifikat Sistem Manajemen Pengamanan Informasi.
            (2) Penyelenggara Sistem Elektronik rendah dapat memiliki Sertifikat Sistem Manajemen Pengamanan Informasi.
            """
        ).strip(),
        "reference_contexts": [
            "Peraturan Menteri Komunikasi dan Informatika Nomor 4 Tahun 2016 tentang Sistem Manajemen Pengamanan Informasi, BAB IV - PENYELENGGARAAN, Bagian Kesatu - Penyelenggara Sistem Elektronik, Pasal 10:\n(1) Penyelenggara Sistem Elektronik strategis dan Penyelenggara Sistem Elektronik tinggi wajib memiliki Sertifikat Sistem Manajemen Pengamanan Informasi.\n(2) Penyelenggara Sistem Elektronik rendah dapat memiliki Sertifikat Sistem Manajemen Pengamanan Informasi."
        ]
    },
    {
        "user_input": "Berdasarkan daftar peraturan teknologi informasi yang ada, subjek apa saja yang dibahas?",
        "reference": (
            """ 
            Berikut adalah daftar topik atau subjek peraturan yang ada:

            1.  ADMINISTRASI DAN TATA USAHA NEGARA
            2.  CIPTA KERJA
            3.  HAK ASASI MANUSIA
            4.  HAK ATAS KEKAYAAN INTELEKTUAL
            5.  ILMU PENGETAHUAN DAN TEKNOLOGI
            6.  INFORMASI PUBLIK
            7.  JABATAN / PROFESI / KEAHLIAN / SERTIFIKASI
            8.  JARINGAN DOKUMENTASI DAN INFORMASI HUKUM / JDIH
            9.  OTONOMI DAERAH DAN PEMERINTAH DAERAH
            10. PARTAI POLITIK DAN PEMILU
            11. PEMILIHAN UMUM DAERAH, DPRD, PEMILIHAN KEPALA DAERAH
            12. PENDIDIKAN
            13. PENGADAAN BARANG / JASA
            14. PERIZINAN, PELAYANAN PUBLIK
            15. PERLINDUNGAN KONSUMEN
            16. PERLINDUNGAN USAHA, PERUSAHAAN, BADAN USAHA, PERDAGANGAN
            17. PERS, POS, DAN PERIKLANAN
            18. PROGRAM, RENCANA PEMBANGUNAN DAN RENCANA KERJA
            19. SISTEM PENGENDALIAN INTERN
            20. STANDAR / PEDOMAN
            21. STRUKTUR ORGANISASI
            22. TELEKOMUNIKASI, INFORMATIKA, SIBER, DAN INTERNET
            """
        ).strip(),
        "reference_contexts": [
            "{'subject': 'ADMINISTRASI DAN TATA USAHA NEGARA'}",
            "{'subject': 'CIPTA KERJA'}",
            "{'subject': 'HAK ASASI MANUSIA'}",
            "{'subject': 'HAK ATAS KEKAYAAN INTELEKTUAL'}",
            "{'subject': 'ILMU PENGETAHUAN DAN TEKNOLOGI'}",
            "{'subject': 'INFORMASI PUBLIK'}",
            "{'subject': 'JABATAN / PROFESI / KEAHLIAN / SERTIFIKASI'}",
            "{'subject': 'JARINGAN DOKUMENTASI DAN INFORMASI HUKUM / JDIH'}",
            "{'subject': 'OTONOMI DAERAH DAN PEMERINTAH DAERAH'}",
            "{'subject': 'PARTAI POLITIK DAN PEMILU'}",
            "{'subject': 'PEMILIHAN UMUM DAERAH, DPRD, PEMILIHAN KEPALA DAERAH'}",
            "{'subject': 'PENDIDIKAN'}",
            "{'subject': 'PENGADAAN BARANG / JASA'}",
            "{'subject': 'PERIZINAN, PELAYANAN PUBLIK'}",
            "{'subject': 'PERLINDUNGAN KONSUMEN'}",
            "{'subject': 'PERLINDUNGAN USAHA, PERUSAHAAN, BADAN USAHA, PERDAGANGAN'}",
            "{'subject': 'PERS, POS, DAN PERIKLANAN'}",
            "{'subject': 'PROGRAM, RENCANA PEMBANGUNAN DAN RENCANA KERJA'}",
            "{'subject': 'SISTEM PENGENDALIAN INTERN'}",
            "{'subject': 'STANDAR / PEDOMAN'}",
            "{'subject': 'STRUKTUR ORGANISASI'}",
            "{'subject': 'TELEKOMUNIKASI, INFORMATIKA, SIBER, DAN INTERNET'}",
        ]
    },
    {
        "user_input": "Apa judul pasal selanjutnya setelah Pasal 39 UU Nomor 11 Tahun 2008?",
        "reference": (
            """
            Pasal selanjutnya setelah Pasal 39 UU Nomor 11 Tahun 2008 adalah Pasal 40. 

            Namun, perlu diperhatikan bahwa terdapat tiga node Pasal 40 yang berbeda, yaitu:
            - Pasal 40 dari UU Nomor 1 Tahun 2024 (status: Berlaku)
            - Pasal 40 dari UU Nomor 11 Tahun 2008 (status: Tidak Berlaku)
            - Pasal 40 dari UU Nomor 19 Tahun 2016 (status: Tidak Berlaku)

            Ini menunjukkan bahwa Pasal 40 dari UU Nomor 11 Tahun 2008 telah diubah oleh UU Nomor 19 Tahun 2016, dan kemudian diubah lagi oleh UU Nomor 1 Tahun 2024. Pasal 40 yang berlaku saat ini adalah Pasal 40 dari UU Nomor 1 Tahun 2024.
            """
        ).strip(),
        "reference_contexts": [
            "{'title': 'Pasal 40 UU Nomor 11 Tahun 2008', 'status': 'Tidak Berlaku'}",
            "{'title': 'Pasal 40 UU Nomor 19 Tahun 2016 tentang Perubahan Atas Undang-Undang Nomor 11 Tahun 2008', 'status': 'Tidak Berlaku'}",
            "{'title': 'Pasal 40 UU Nomor 1 Tahun 2024 tentang Perubahan Kedua atas Undang-Undang Nomor 11 Tahun 2008', 'status': 'Berlaku'}",
        ]
    },
    {
        "user_input": "Apa pasal sebelum Pasal 41 UU Nomor 11 Tahun 2008?",
        "reference": (
            """
            Pasal sebelum Pasal 41 UU Nomor 11 Tahun 2008 adalah Pasal 40A.

            Namun, perlu diperhatikan bahwa terdapat tiga node Pasal 40 yang berbeda, yaitu:
            - Pasal 40A dari UU Nomor 1 Tahun 2024 (status: Berlaku)
            - Pasal 40 dari UU Nomor 11 Tahun 2008 (status: Tidak Berlaku)
            - Pasal 40 dari UU Nomor 19 Tahun 2016 (status: Tidak Berlaku)

            Ini menunjukkan bahwa Pasal 40 dari UU Nomor 11 Tahun 2008 telah diubah oleh UU Nomor 19 Tahun 2016, dan kemudian ditambahkan Pasal 40A oleh UU Nomor 1 Tahun 2024. Pasal 40A yang berlaku saat ini adalah Pasal 40A dari UU Nomor 1 Tahun 2024.
            """
        ).strip(),
        "reference_contexts": [
            "{'title': 'Pasal 40 UU Nomor 11 Tahun 2008', 'status': 'Tidak Berlaku'}",
            "{'title': 'Pasal 40 UU Nomor 19 Tahun 2016 tentang Perubahan Atas Undang-Undang Nomor 11 Tahun 2008', 'status': 'Tidak Berlaku'}",
            "'title': 'Pasal 40A UU Nomor 1 Tahun 2024 tentang Perubahan Kedua atas Undang-Undang Nomor 11 Tahun 2008', 'status': 'Berlaku'}",
        ]
    },
    {
        "user_input": "Apakah Pasal 40 UU Nomor 11 Tahun 2008 pernah diamandemen? Dan jika pernah, apa judul pasal amandemennya?",
        "reference": (
            """
            Ya, Pasal 40 UU Nomor 11 Tahun 2008 pernah diamandemen.

            Pasal amandemennya adalah:

            1.  Pasal 40 dari UU Nomor 1 Tahun 2024 tentang Perubahan Kedua atas Undang-Undang Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik.
            2.  Pasal 40 dari UU Nomor 19 Tahun 2016 tentang Perubahan Atas Undang-Undang Nomor 11 Tahun 2008 Tentang Informasi Dan Transaksi Elektronik.

            Perlu diperhatikan bahwa Pasal 40 dari UU Nomor 11 Tahun 2008 telah diubah oleh UU Nomor 19 Tahun 2016, dan kemudian diubah lagi oleh UU Nomor 1 Tahun 2024. Pasal 40 yang berlaku saat ini adalah Pasal 40 dari UU Nomor 1 Tahun 2024
            """
        ).strip(),
        "reference_contexts": [
            "{'title': 'Pasal 40 UU Nomor 19 Tahun 2016 tentang Perubahan Atas Undang-Undang Nomor 11 Tahun 2008', 'status': 'Tidak Berlaku'}",
            "{'title': 'Pasal 40 UU Nomor 1 Tahun 2024 tentang Perubahan Kedua atas Undang-Undang Nomor 11 Tahun 2008', 'status': 'Berlaku'}",
        ]
    },
    {
        "user_input": "Apa saja pertimbangan utama atau latar belakang yang mendasari pembuatan PERMENKOMINFO Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat?",
        "reference": (
            """
            Pertimbangan utama yang mendasari pembuatan PERMENKOMINFO Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat adalah:

            1. Kewajiban Penyelenggara Telekomunikasi: Pasal 20 Undang-Undang Nomor 36 Tahun 1999 tentang Telekomunikasi mewajibkan setiap penyelenggara telekomunikasi untuk memberikan prioritas pada pengiriman, penyaluran, dan penyampaian informasi penting terkait keamanan negara, keselamatan jiwa manusia dan harta benda, bencana alam, marabahaya, dan/atau wabah penyakit.

            2. Optimalisasi Layanan Nomor Tunggal Panggilan Darurat: Perlunya optimalisasi layanan Nomor Tunggal Panggilan Darurat di Indonesia untuk mendukung pengiriman, penyaluran, dan penyampaian informasi penting tersebut, sehingga penanganan keadaan darurat dapat dilaksanakan secara terpadu.
            """
        ).strip(),
        "reference_contexts": [
            "{'text': 'Daftar Latar Belakang dari Peraturan Menteri Komunikasi dan Informatika Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat:\na. bahwa ketentuan Pasal 20 Undang-Undang Nomor 36 Tahun 1999 tentang Telekomunikasi menyatakan setiap penyelenggara telekomunikasi wajib memberikan prioritas untuk pengiriman, penyaluran, dan penyampaian informasi penting yang menyangkut keamanan negara, keselamatan jiwa manusia dan harta benda, bencana alam, marabahaya, dan/atau wabah penyakit;\nb. bahwa layanan Nomor Tunggal Panggilan Darurat di Indonesia perlu lebih dioptimalkan untuk mendukung pengiriman, penyaluran, dan penyampaian informasi penting yang menyangkut keamanan negara, keselamatan jiwa manusia dan harta benda, bencana alam, marabahaya, dan/atau wabah penyakit sebagaimana dimaksud pada huruf a, sehingga penanganan keadaan darurat dapat dilaksanakan secara terpadu;\nc. bahwa berdasarkan pertimbangan sebagaimana dimaksud dalam huruf a dan huruf b, perlu menetapkan Peraturan Menteri Komunikasi dan Informatika tentang Layanan Nomor Tunggal Panggilan Darurat;'}",
        ]
    },
    {
        "user_input": "Apa saja dasar hukum yang mendasari pembuatan PERMENKOMINFO Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat?",
        "reference": (
            """
            Berdasarkan hasil kueri, dasar hukum yang mendasari pembuatan PERMENKOMINFO Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat adalah:

            1.  Undang-Undang Nomor 36 Tahun 1999 tentang Telekomunikasi (Lembaran Negara Republik Indonesia Tahun 1999 Nomor 154, Tambahan Lembaran Negara Republik Indonesia Nomor 3881);
            2.  Undang-Undang Nomor 24 Tahun 2007 tentang Penanggulangan Bencana (Lembaran Negara Republik Indonesia Tahun 2007 Nomor 66, Tambahan Lembaran Negara Republik Indonesia Nomor 4723);
            3.  Undang-Undang Nomor 39 Tahun 2008 tentang Kementerian Negara (Lembaran Negara Republik Indonesia Tahun 2008 Nomor 166, Tambahan Lembaran Negara Republik Indonesia Nomor 4916);
            4.  Undang-Undang Nomor 23 Tahun 2014 tentang Pemerintah Daerah (Lembaran Negara Republik Indonesia Tahun 2014 Nomor 244, Tambahan Lembaran Negara Republik Indonesia Nomor 5587);
            5.  Undang-Undang Nomor 30 Tahun 2014 tentang Administrasi Pemerintahan (Lembaran Negara Republik Indonesia Tahun 2014 Nomor: 292, Tambahan Lembaran Negara Republik Indonesia Nomor: 5601);
            6.  Peraturan Pemerintah Nomor 52 Tahun 2000 tentang Penyelenggaraan Telekomunikasi (Lembaran Negara Republik Indonesia Tahun 2000 Nomor 107, Tambahan Lembaran Negara Republik Indonesia Nomor 3980);
            7.  Peraturan Pemerintah Nomor 6 Tahun 2006 tentang Pengelolaan Barang Milik Negara/Daerah (Lembaran Negara Republik Indonesia Tahun 2006 Nomor 20, Tambahan Lembaran Negara Republik Indonesia Nomor 4609);
            8.  Peraturan Presiden Nomor 7 Tahun 2015 tentang Organisasi Kementerian Negara (Lembaran Negara Republik Indonesia Tahun 2015 Nomor 8);
            9.  Peraturan Presiden Nomor 54 Tahun 2015 tentang Kementerian Komunikasi dan Informatika (Lembaran Negara Republik Indonesia Tahun 2015 Nomor 96);
            10. Peraturan Menteri Keuangan Nomor 96/PMK.06/2007 tentang Tata Cara Pelaksanaan Penggunaan, Pemanfaatan, Penghapusan, dan Pemindahtanganan Barang Milik Negara;
            11. Peraturan Menteri Komunikasi dan Informatika Nomor: 01/PER/M.KOMINFO/01/2010 tentang Penyelenggaraan Jaringan Telekomunikasi sebagaimana telah beberapa kali diubah terakhir dengan Peraturan Menteri Komunikasi dan Informatika Nomor 7 Tahun 2015 tentang Perubahan Kedua atas Peraturan Menteri Komunikasi dan Informatika Nomor: 01/PER/M.KOMINFO/01/2010 tentang Penyelenggaraan Jaringan Telekomunikasi (Berita Negara Republik Indonesia Tahun 2015 Nomor 250);
            12. Peraturan Menteri Komunikasi dan Informatika Nomor 18 Tahun 2014 tentang Sertifikasi Alat dan Perangkat Telekomunikasi sebagaimana telah diubah dengan Peraturan Menteri Komunikasi dan Informatika Nomor 1 Tahun 2015 tentang Perubahan atas Peraturan Menteri Komunikasi dan Informatika Nomor 18 Tahun 2014 tentang Sertifikasi Alat dan Perangkat Telekomunikasi (Berita Negara Republik Indonesia Tahun 2014 Nomor 882);
            13. Peraturan Menteri Komunikasi dan Informatika Nomor 1 Tahun 2016 tentang Organisasi dan Tata Kerja Kementerian Komunikasi dan Informatika (Berita Negara Republik Indonesia Tahun 2016 Nomor 103);
            14. Keputusan Menteri Perhubungan Nomor: KM.4 Tahun 2001 tentang Penetapan Rencana Dasar Teknis Nasional 2000 (Fundamental Technical Plan National 2000) Pembangunan Telekomunikasi Nasional sebagaimana telah beberapa kali diubah, terakhir dengan Peraturan Menteri Komunikasi dan Informatika Nomor 17 Tahun 2014 tentang Perubahan Ketujuh atas Keputusan Menteri Perhubungan Nomor: KM.4 Tahun 2001 tentang Penetapan Rencana Dasar Teknis Nasional 2000 (Fundamental Technical Plan National 2000) Pembangunan Telekomunikasi Nasional (Berita Negara Republik Indonesia Tahun 2014 Nomor 770);
            """
        ).strip(),
        "reference_contexts": [
            "'text': 'Daftar Dasar Hukum dari Peraturan Menteri Komunikasi dan Informatika Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat:\n(1) Undang-Undang Nomor 36 Tahun 1999 tentang Telekomunikasi (Lembaran Negara Republik Indonesia Tahun 1999 Nomor 154, Tambahan Lembaran Negara Republik Indonesia Nomor 3881);\n(2) Undang-Undang Nomor 24 Tahun 2007 tentang Penanggulangan Bencana (Lembaran Negara Republik Indonesia Tahun 2007 Nomor 66, Tambahan Lembaran Negara Republik Indonesia Nomor 4723);\n(3) Undang-Undang Nomor 39 Tahun 2008 tentang Kementerian Negara (Lembaran Negara Republik Indonesia Tahun 2008 Nomor 166, Tambahan Lembaran Negara Republik Indonesia Nomor 4916);\n(4) Undang-Undang Nomor 23 Tahun 2014 tentang Pemerintah Daerah (Lembaran Negara Republik Indonesia Tahun 2014 Nomor 244, Tambahan Lembaran Negara Republik Indonesia Nomor 5587);\n(5) Undang-Undang Nomor 30 Tahun 2014 tentang Administrasi Pemerintahan (Lembaran Negara Republik Indonesia Tahun 2014 Nomor: 292, Tambahan Lembaran Negara Republik Indonesia Nomor: 5601);\n(6) Peraturan Pemerintah Nomor 52 Tahun 2000 tentang Penyelenggaraan Telekomunikasi (Lembaran Negara Republik Indonesia Tahun 2000 Nomor 107, Tambahan Lembaran Negara Republik Indonesia Nomor 3980);\n(7) Peraturan Pemerintah Nomor 6 Tahun 2006 tentang Pengelolaan Barang Milik Negara/Daerah (Lembaran Negara Republik Indonesia Tahun 2006 Nomor 20, Tambahan Lembaran Negara Republik Indonesia Nomor 4609);\n(8) Peraturan Presiden Nomor 7 Tahun 2015 tentang Organisasi Kementerian Negara (Lembaran Negara Republik Indonesia Tahun 2015 Nomor 8);\n(9) Peraturan Presiden Nomor 54 Tahun 2015 tentang Kementerian Komunikasi dan Informatika (Lembaran Negara Republik Indonesia Tahun 2015 Nomor 96);\n(10) Peraturan Menteri Keuangan Nomor 96/PMK.06/2007 tentang Tata Cara Pelaksanaan Penggunaan, Pemanfaatan, Penghapusan, dan Pemindahtanganan Barang Milik Negara;\n(11) Peraturan Menteri Komunikasi dan Informatika Nomor: 01/PER/M.KOMINFO/01/2010 tentang Penyelenggaraan Jaringan Telekomunikasi sebagaimana telah beberapa kali diubah terakhir dengan Peraturan Menteri Komunikasi dan Informatika Nomor 7 Tahun 2015 tentang Perubahan Kedua atas Peraturan Menteri Komunikasi dan Informatika Nomor: 01/PER/M.KOMINFO/01/2010 tentang Penyelenggaraan Jaringan Telekomunikasi (Berita Negara Republik Indonesia Tahun 2015 Nomor 250);\n(12) Peraturan Menteri Komunikasi dan Informatika Nomor 18 Tahun 2014 tentang Sertifikasi Alat dan Perangkat Telekomunikasi sebagaimana telah diubah dengan Peraturan Menteri Komunikasi dan Informatika Nomor 1 Tahun 2015 tentang Perubahan atas Peraturan Menteri Komunikasi dan Informatika Nomor 18 Tahun 2014 tentang Sertifikasi Alat dan Perangkat Telekomunikasi (Berita Negara Republik Indonesia Tahun 2014 Nomor 882);\n(13) Peraturan Menteri Komunikasi dan Informatika Nomor 1 Tahun 2016 tentang Organisasi dan Tata Kerja Kementerian Komunikasi dan Informatika (Berita Negara Republik Indonesia Tahun 2016 Nomor 103);\n(14) Keputusan Menteri Perhubungan Nomor: KM.4 Tahun 2001 tentang Penetapan Rencana Dasar Teknis Nasional 2000 (Fundamental Technical Plan National 2000) Pembangunan Telekomunikasi Nasional sebagaimana telah beberapa kali diubah, terakhir dengan Peraturan Menteri Komunikasi dan Informatika Nomor 17 Tahun 2014 tentang Perubahan Ketujuh atas Keputusan Menteri Perhubungan Nomor: KM.4 Tahun 2001 tentang Penetapan Rencana Dasar Teknis Nasional 2000 (Fundamental Technical Plan National 2000) Pembangunan Telekomunikasi Nasional (Berita Negara Republik Indonesia Tahun 2014 Nomor 770);'}",
        ]
    },
    {
        "user_input": "Apa saja judul pasal yang dirujuk oleh Pasal 25 PERMENKOMINFO No. 4 Tahun 2016?",
        "reference": (
            """
            Pasal 25 PERMENKOMINFO No. 4 Tahun 2016 merujuk ke pasal-pasal berikut:
            - Pasal 10 PERMENKOMINFO No. 4 Tahun 2016
            - Pasal 22 PERMENKOMINFO No. 4 Tahun 2016
            """
        ).strip(),
        "reference_contexts": [
            "{'title': 'Pasal 10 PERMENKOMINFO Nomor 4 Tahun 2016', 'status': 'Berlaku'}",
            "{'title': 'Pasal 22 PERMENKOMINFO Nomor 4 Tahun 2016', 'status': 'Berlaku'}",
        ]
    },
    {
        "user_input": "Apa saja judul pasal di peraturan lain yang mirip dengan Pasal 46 PP No. 52 Tahun 2000?",
        "reference": (
            """
            Pasal-pasal di peraturan lain yang mirip dengan Pasal 46 PP No. 52 Tahun 2000 adalah:

            *   Pasal 2 PERMENKOMINFO Nomor 12 Tahun 2018
            *   Pasal 10 PERMENKOMINFO Nomor 13 Tahun 2019
            *   Pasal 131 PERMENKOMINFO Nomor 5 Tahun 2021
            *   Pasal 196 PERMENKOMINFO Nomor 5 Tahun 2021
            *   Pasal 30 UU Nomor 36 Tahun 1999
            """
        ).strip(),
        "reference_contexts": [
            "{'title': 'Pasal 2 PERMENKOMINFO Nomor 12 Tahun 2018'}",
            "{'title': 'Pasal 10 PERMENKOMINFO Nomor 13 Tahun 2019'}",
            "{'title': 'Pasal 131 PERMENKOMINFO Nomor 5 Tahun 2021'}",
            "{'title': 'Pasal 196 PERMENKOMINFO Nomor 5 Tahun 2021'}",
            "{'title': 'Pasal 30 UU Nomor 36 Tahun 1999'}",
        ]
    },
    {
        "user_input": "Pasal 19 dalam Peraturan Kominfo Nomor 13 Tahun 2019 saat ini masih berlaku atau sudah dicabut?",
        "reference": (
            """
            Pasal 19 dalam Peraturan Kominfo Nomor 13 Tahun 2019 saat ini sudah dicabut atau tidak berlaku.
            """
        ).strip(),
        "reference_contexts": [
            "{'article_status': 'Tidak Berlaku'}",
        ]
    },
    {
        "user_input": "Peraturan Pemerintah (PP) apa saja yang dikeluarkan pada tahun 2019 yang secara spesifik mengatur atau terkait dengan bidang teknologi informasi, komunikasi, siber, atau data?",
        "reference": (
            """
            Berdasarkan hasil kueri, Peraturan Pemerintah (PP) yang dikeluarkan pada tahun 2019 yang mengatur bidang teknologi informasi, komunikasi, siber, atau data adalah:
            1.  Peraturan Pemerintah (PP) Nomor 71 Tahun 2019 tentang Penyelenggaraan Sistem dan Transaksi Elektronik
            2.  Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 tentang Perdagangan Melalui Sistem Elektronik
            """
        ).strip(),
        "reference_contexts": [
            "{'title': 'Peraturan Pemerintah (PP) Nomor 71 Tahun 2019 tentang Penyelenggaraan Sistem dan Transaksi Elektronik'}",
            "{'title': 'Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 tentang Perdagangan Melalui Sistem Elektronik'}",
        ]
    },
    {
        "user_input": "Peraturan apa saja yang dikeluarkan pada rentang tahun 2008 hingga 2010 yang mengatur bidang teknologi informasi, komunikasi, siber, atau data?",
        "reference": (
            """
            Berdasarkan hasil kueri, berikut adalah peraturan yang dikeluarkan pada rentang tahun 2008 hingga 2010 yang mengatur bidang teknologi informasi, komunikasi, siber, atau data:
            1. Peraturan Menteri Komunikasi dan Informatika Nomor 41/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Penilaian Pencapaian Tingkat Komponen Dalam Negeri pada Penyelenggaraan Telekomunikasi
            2. Peraturan Menteri Komunikasi dan Informatika Nomor 42/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Memperoleh Izin bagi Lembaga Penyiaran Asing yang Melakukan Kegiatan Peliputan di Indonesia
            3. Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik
            """
        ).strip(),
        "reference_contexts": [
            "{'title': 'Peraturan Menteri Komunikasi dan Informatika Nomor 41/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Penilaian Pencapaian Tingkat Komponen Dalam Negeri pada Penyelenggaraan Telekomunikasi'}",
            "{'title': 'Peraturan Menteri Komunikasi dan Informatika Nomor 42/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Memperoleh Izin bagi Lembaga Penyiaran Asing yang Melakukan Kegiatan Peliputan di Indonesia'}",
            "{'title': 'Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik'}",
        ]
    },
    {
        "user_input": "Pasal apa saja dari PERMENKOMINFO Nomor 26 Tahun 2007 yang sudah tidak berlaku?",
        "reference": (
            """
            Pasal-pasal dari PERMENKOMINFO Nomor 26 Tahun 2007 yang sudah tidak berlaku adalah Pasal 1, 3, 4, 7, 9, 11, 12, 13, dan 17.
            """
        ).strip(),
        "reference_contexts": [
            "{'title': 'Pasal 1 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
            "{'title': 'Pasal 3 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
            "{'title': 'Pasal 4 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
            "{'title': 'Pasal 7 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
            "{'title': 'Pasal 9 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
            "{'title': 'Pasal 11 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
            "{'title': 'Pasal 12 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
            "{'title': 'Pasal 13 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
            "{'title': 'Pasal 17 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
        ]
    },
])

In [15]:
text2cypher_eval_workflow_result = text2cypher_eval_workflow(
    evaluation_dataset[5:],
    neo4j_graph=neo4j_graph,
    cypher_llm=gemini_llm,
    qa_llm=gemini_llm,
    embedder_model=embedder_model,
    verbose=True
)

evaluation_dataset_text2cypher, generated_cypher_results = text2cypher_eval_workflow_result

Running text2cypher_retriever on evaluation dataset: 100%|██████████| 15/15 [00:15<00:00,  1.05s/it]


In [16]:
num = 0
for data in evaluation_dataset_text2cypher:
    for context in data.retrieved_contexts:
        num += 1
    display(data.model_dump())
print(num)

{'user_input': 'Apa judul dari Peraturan Pemerintah (PP) Nomor 80 Tahun 2019?',
 'retrieved_contexts': ["{'title': 'Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 tentang Perdagangan Melalui Sistem Elektronik'}"],
 'reference_contexts': ['Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 tentang Perdagangan Melalui Sistem Elektronik'],
 'response': None,
 'multi_responses': None,
 'reference': 'Judul dari Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 adalah Perdagangan Melalui Sistem Elektronik.',
 'rubrics': None}

{'user_input': 'Apa isi Pasal 2 Peraturan Pemerintah (PP) Nomor 80 Tahun 2019?',
 'retrieved_contexts': ["{'text': 'Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 tentang Perdagangan Melalui Sistem Elektronik, BAB II - LINGKUP PENGATURAN DAN PRINSIP PERDAGANGAN MELALUI SISTEM ELEKTRONIK, Pasal 2:\\nLingkup pengaturan Perdagangan Melalui Sistem Elektronik meliputi:\\na. pihak yang melakukan PMSE;\\nb. persyaratan dalam PMSE;\\nc. penyelenggaraan PMSE;\\nd. kewajiban Pelaku Usaha;\\ne. bukti transaksi PMSE;\\nf. Iklan Elektronik;\\ng. Penawaran Secara Elektronik, Penerimaan Secara Elektronik, dan Konfirmasi Elektronik;\\nh. Kontrak Elektronik;\\ni. perlindungan terhadap data pribadi;\\nj. pembayaran dalam PMSE;\\nk. pengiriman Barang dan Jasa dalam PMSE;\\n1. penukaran Barang atau Jasa dan pembatalan pembelian dalam PMSE;\\nm. penyelesaian sengketa dalam PMSE; dan\\nn. pembinaan dan pengawasan.'}"],
 'reference_contexts': ['Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 tentang Perdagangan

{'user_input': 'Apa isi Pasal 10 Peraturan Menteri Komunikasi dan Informatika (PERMENKOMINFO) Nomor 4 Tahun 2016?',
 'retrieved_contexts': ["{'text': 'Peraturan Menteri Komunikasi dan Informatika Nomor 4 Tahun 2016 tentang Sistem Manajemen Pengamanan Informasi, BAB IV - PENYELENGGARAAN, Bagian Kesatu - Penyelenggara Sistem Elektronik, Pasal 10:\\n(1) Penyelenggara Sistem Elektronik strategis dan Penyelenggara Sistem Elektronik tinggi wajib memiliki Sertifikat Sistem Manajemen Pengamanan Informasi.\\n(2) Penyelenggara Sistem Elektronik rendah dapat memiliki Sertifikat Sistem Manajemen Pengamanan Informasi.'}"],
 'reference_contexts': ['Peraturan Menteri Komunikasi dan Informatika Nomor 4 Tahun 2016 tentang Sistem Manajemen Pengamanan Informasi, BAB IV - PENYELENGGARAAN, Bagian Kesatu - Penyelenggara Sistem Elektronik, Pasal 10:\n(1) Penyelenggara Sistem Elektronik strategis dan Penyelenggara Sistem Elektronik tinggi wajib memiliki Sertifikat Sistem Manajemen Pengamanan Informasi.\n(2) P

{'user_input': 'Berdasarkan daftar peraturan teknologi informasi yang ada, subjek apa saja yang dibahas?',
 'retrieved_contexts': ["{'title': 'TELEKOMUNIKASI, INFORMATIKA, SIBER, DAN INTERNET'}",
  "{'title': 'PERIZINAN, PELAYANAN PUBLIK'}",
  "{'title': 'PERS, POS, DAN PERIKLANAN'}",
  "{'title': 'JABATAN / PROFESI / KEAHLIAN / SERTIFIKASI'}",
  "{'title': 'JARINGAN DOKUMENTASI DAN INFORMASI HUKUM / JDIH'}",
  "{'title': 'PARTAI POLITIK DAN PEMILU'}",
  "{'title': 'PEMILIHAN UMUM DAERAH, DPRD, PEMILIHAN KEPALA DAERAH'}",
  "{'title': 'ILMU PENGETAHUAN DAN TEKNOLOGI'}",
  "{'title': 'HAK ATAS KEKAYAAN INTELEKTUAL'}",
  "{'title': 'OTONOMI DAERAH DAN PEMERINTAH DAERAH'}",
  "{'title': 'STRUKTUR ORGANISASI'}",
  "{'title': 'STANDAR / PEDOMAN'}",
  "{'title': 'PERLINDUNGAN KONSUMEN'}",
  "{'title': 'PENDIDIKAN'}",
  "{'title': 'PROGRAM, RENCANA PEMBANGUNAN DAN RENCANA KERJA'}",
  "{'title': 'ADMINISTRASI DAN TATA USAHA NEGARA'}",
  "{'title': 'SISTEM PENGENDALIAN INTERN'}",
  "{'title': '

{'user_input': 'Apa judul pasal selanjutnya setelah Pasal 39 UU Nomor 11 Tahun 2008?',
 'retrieved_contexts': ["{'title': 'Pasal 40 UU Nomor 11 Tahun 2008'}",
  "{'title': 'Pasal 40 UU Nomor 19 Tahun 2016 tentang Perubahan Atas Undang-Undang Nomor 11 Tahun 2008'}",
  "{'title': 'Pasal 40 UU Nomor 1 Tahun 2024 tentang Perubahan Kedua atas Undang-Undang Nomor 11 Tahun 2008'}"],
 'reference_contexts': ["{'title': 'Pasal 40 UU Nomor 11 Tahun 2008', 'status': 'Tidak Berlaku'}",
  "{'title': 'Pasal 40 UU Nomor 19 Tahun 2016 tentang Perubahan Atas Undang-Undang Nomor 11 Tahun 2008', 'status': 'Tidak Berlaku'}",
  "{'title': 'Pasal 40 UU Nomor 1 Tahun 2024 tentang Perubahan Kedua atas Undang-Undang Nomor 11 Tahun 2008', 'status': 'Berlaku'}"],
 'response': None,
 'multi_responses': None,
 'reference': 'Pasal selanjutnya setelah Pasal 39 UU Nomor 11 Tahun 2008 adalah Pasal 40. \n\n            Namun, perlu diperhatikan bahwa terdapat tiga node Pasal 40 yang berbeda, yaitu:\n            - Pasal 4

...

{'user_input': 'Pasal 19 dalam Peraturan Kominfo Nomor 13 Tahun 2019 saat ini masih berlaku atau sudah dicabut?',
 'retrieved_contexts': ["{'status': 'Tidak Berlaku'}"],
 'reference_contexts': ["{'article_status': 'Tidak Berlaku'}"],
 'response': None,
 'multi_responses': None,
 'reference': 'Pasal 19 dalam Peraturan Kominfo Nomor 13 Tahun 2019 saat ini sudah dicabut atau tidak berlaku.',
 'rubrics': None}

{'user_input': 'Peraturan Pemerintah (PP) apa saja yang dikeluarkan pada tahun 2019 yang secara spesifik mengatur atau terkait dengan bidang teknologi informasi, komunikasi, siber, atau data?',
 'retrieved_contexts': ["{'title': 'Peraturan Pemerintah (PP) Nomor 71 Tahun 2019 tentang Penyelenggaraan Sistem dan Transaksi Elektronik'}",
  "{'title': 'Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 tentang Perdagangan Melalui Sistem Elektronik'}"],
 'reference_contexts': ["{'title': 'Peraturan Pemerintah (PP) Nomor 71 Tahun 2019 tentang Penyelenggaraan Sistem dan Transaksi Elektronik'}",
  "{'title': 'Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 tentang Perdagangan Melalui Sistem Elektronik'}"],
 'response': None,
 'multi_responses': None,
 'reference': 'Berdasarkan hasil kueri, Peraturan Pemerintah (PP) yang dikeluarkan pada tahun 2019 yang mengatur bidang teknologi informasi, komunikasi, siber, atau data adalah:\n            1.  Peraturan Pemerintah (PP) Nomor 71 Tahun 2019 tentang Penyel

{'user_input': 'Peraturan apa saja yang dikeluarkan pada rentang tahun 2008 hingga 2010 yang mengatur bidang teknologi informasi, komunikasi, siber, atau data?',
 'retrieved_contexts': ["{'title': 'Peraturan Menteri Komunikasi dan Informatika Nomor 41/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Penilaian Pencapaian Tingkat Komponen Dalam Negeri pada Penyelenggaraan Telekomunikasi'}",
  "{'title': 'Peraturan Menteri Komunikasi dan Informatika Nomor 42/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Memperoleh Izin bagi Lembaga Penyiaran Asing yang Melakukan Kegiatan Peliputan di Indonesia'}",
  "{'title': 'Peraturan Menteri Komunikasi dan Informatika Nomor 1/PER/M.KOMINFO/1/2010 Tahun 2010 tentang Penyelenggaraan Jaringan Telekomunikasi'}",
  "{'title': 'Peraturan Menteri Komunikasi dan Informatika Nomor 16/PER/M.KOMINFO/10/2010 Tahun 2010 tentang Perubahan atas Peraturan Menteri Komunikasi dan Informatika Nomor 26/PER/M.KOMINFO/5/2007 tentang Pengamanan Pemanfaatan Jaringan T

{'user_input': 'Pasal apa saja dari PERMENKOMINFO Nomor 26 Tahun 2007 yang sudah tidak berlaku?',
 'retrieved_contexts': ["{'title': 'Pasal 1 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
  "{'title': 'Pasal 3 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
  "{'title': 'Pasal 4 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
  "{'title': 'Pasal 7 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
  "{'title': 'Pasal 9 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
  "{'title': 'Pasal 11 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
  "{'title': 'Pasal 12 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
  "{'title': 'Pasal 13 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
  "{'title': 'Pasal 17 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}"],
 'reference_contexts': ["{'title': 'Pasal 1 PERMENKOMINFO Nomor 26/PER/M.KOMINFO/5/2007 Tahun 2007'}",
  "{'title': 'Pasal 3 PERMENKOMINFO Nomor 2

60


In [9]:
generated_cypher_results

["cypher \nMATCH (r:Regulation)\nWHERE r.type = 'PP' AND r.number = 80 AND r.year = 2019\nRETURN r.title AS title\n",
 "cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'PP' AND r.number = 80 AND r.year = 2019 AND a.number = '2'\nRETURN a.text AS text\n",
 "cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'PERMENKOMINFO' AND r.number = 4 AND r.year = 2016 AND a.number = '10'\nRETURN a.text AS text\n",
 'cypher \nMATCH (r:Regulation)-[:HAS_SUBJECT]->(s:Subject)\nRETURN DISTINCT s.title AS title\n',
 "cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:NEXT_ARTICLE]->(next_article)\nWHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '39'\nRETURN next_article.title AS title\n",
 "cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:PREVIOUS_ARTICLE]->(previous_article:Article)\nWHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '41'\nRETURN previous_article.text AS text\n",
 "cypher \n

In [10]:
from ragas import evaluate
from dataclasses import dataclass, field
from ragas.metrics.base import MetricType
from typing import (
    Dict,
    Set
)
from ragas.metrics import AspectCritic
from ragas.metrics import SimpleCriteriaScore
from ragas.metrics import NonLLMContextRecall
from ragas.metrics import NonLLMContextPrecisionWithReference
from ragas.metrics import LLMContextRecall
from ragas.metrics import LLMContextPrecisionWithReference
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper



gemini_llm_evaluator = LangchainLLMWrapper(gemini_llm)
gpt_llm_evaluator = LangchainLLMWrapper(gpt_llm)
embedder_evaluator = LangchainEmbeddingsWrapper(embedder_model)

llm_context_recall = LLMContextRecall()
llm_context_precision_with_ref = LLMContextPrecisionWithReference()

non_llm_context_recall = NonLLMContextRecall(threshold=0.5)
non_llm_context_precision = NonLLMContextPrecisionWithReference(threshold=0.5)

required_columns: Dict[MetricType, Set[str]] = {
    MetricType.SINGLE_TURN: {
        "user_input",
        "reference",
        "retrieved_contexts",
    }
}

# Binary
aspect_critic = AspectCritic(
    name="kesesuaian_tipe_1",
    definition="Apakah data yang tersedia di `retrieved_contexts` bisa digunakan untuk menjawab `user_input` untuk menghasilkan jawaban `reference`",
    required_columns=required_columns,
    # strictness=2  # default = 1
)

# Integer
simple_criteria = SimpleCriteriaScore(
    name="kesesuaian_tipe_2",
    definition="Beri nilai 0-100, seberapa baik `retrieved_contexts` bisa digunakan untuk menjawab `user_input` untuk menghasilkan jawaban `reference`",
    required_columns=required_columns,
    # strictness=2  # default = 1
)

# TODO: Perbaiki argumen `definition` nya

# """
# ## Tugas
# Beri nilai 0-100, seberapa baik `retrieved_contexts` bisa digunakan untuk
# menjawab `user_input` untuk menghasilkan jawaban `reference`.

# ## Informasi Tambahan: 
# - `retrieved_context`: Adalah data yang berhasil diambil melalui mekanisme
#     Text2Cypher. Data tersebut merupakan jawaban dalam bentuk 
# """

In [11]:
evaluation_dataset_text2cypher[10:11][0].model_dump()

{'user_input': 'Apa saja judul pasal di peraturan lain yang mirip dengan Pasal 46 PP No. 52 Tahun 2000?',
 'retrieved_contexts': ["{'title': 'Pasal 2 PERMENKOMINFO Nomor 12 Tahun 2018'}",
  "{'title': 'Pasal 10 PERMENKOMINFO Nomor 13 Tahun 2019'}",
  "{'title': 'Pasal 131 PERMENKOMINFO Nomor 5 Tahun 2021'}",
  "{'title': 'Pasal 196 PERMENKOMINFO Nomor 5 Tahun 2021'}",
  "{'title': 'Pasal 30 UU Nomor 36 Tahun 1999'}"],
 'reference_contexts': ["{'title': 'Pasal 2 PERMENKOMINFO Nomor 12 Tahun 2018'}",
  "{'title': 'Pasal 10 PERMENKOMINFO Nomor 13 Tahun 2019'}",
  "{'title': 'Pasal 131 PERMENKOMINFO Nomor 5 Tahun 2021'}",
  "{'title': 'Pasal 196 PERMENKOMINFO Nomor 5 Tahun 2021'}",
  "{'title': 'Pasal 30 UU Nomor 36 Tahun 1999'}"],
 'response': None,
 'multi_responses': None,
 'reference': 'Pasal-pasal di peraturan lain yang mirip dengan Pasal 46 PP No. 52 Tahun 2000 adalah:\n\n            *   Pasal 2 PERMENKOMINFO Nomor 12 Tahun 2018\n            *   Pasal 10 PERMENKOMINFO Nomor 13 Tahun 

In [ ]:
results = evaluate(
    dataset=evaluation_dataset_text2cypher[3:7],
    metrics=[
        llm_context_recall,
        non_llm_context_recall,
        llm_context_precision_with_ref,
        non_llm_context_precision,
        # aspect_critic,
        # simple_criteria
    ],
    llm=gemini_llm_evaluator,
    # embeddings=embedder_evaluator
)

results

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


{'context_recall': 1.0000, 'non_llm_context_recall': 0.7500, 'llm_context_precision_with_reference': 0.7778, 'non_llm_context_precision_with_reference': 0.7500}

In [ ]:
# ISI CONTEXT LANGSUNG JADI 1 STRING DAN DIMASUKKAN KE DALAM LIST (SKIP QA LLM)
# HASILNYA BUKAN YANG DI BAWAH (KEPENCET RUN SKENARIO LAIN), POKOKNYA HASILNYA SEINGETKU KURANG BAGUS
df_gpt_eval_result = results.to_pandas()
df_gpt_eval_result["generated_cypher"] = generated_cypher_results
df_gpt_eval_result

,user_input,retrieved_contexts,reference_contexts,reference,non_llm_context_recall,non_llm_context_precision_with_reference,generated_cypher
0,Apa judul dari Peraturan Pemerintah (PP) Nomor...,[{'title': 'Peraturan Pemerintah (PP) Nomor 80...,[Peraturan Pemerintah (PP) Nomor 80 Tahun 2019...,Judul dari Peraturan Pemerintah (PP) Nomor 80 ...,1.0,1.0,cypher \nMATCH (r:Regulation)\nWHERE r.type = ...
1,Apa isi Pasal 2 Peraturan Pemerintah (PP) Nomo...,[{'text': 'Peraturan Pemerintah (PP) Nomor 80 ...,[Peraturan Pemerintah (PP) Nomor 80 Tahun 2019...,Isi Pasal 2 Peraturan Pemerintah (PP) Nomor 80...,1.0,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
2,Apa isi Pasal 10 Peraturan Menteri Komunikasi ...,[{'text': 'Peraturan Menteri Komunikasi dan In...,[Peraturan Menteri Komunikasi dan Informatika ...,Isi Pasal 10 Peraturan Menteri Komunikasi dan ...,1.0,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
3,Berdasarkan daftar peraturan teknologi informa...,"[{'Subject': 'TELEKOMUNIKASI, INFORMATIKA, SIB...",[{'subject': 'ADMINISTRASI DAN TATA USAHA NEGA...,Berikut adalah daftar topik atau subjek peratu...,1.0,1.0,cypher \nMATCH (s:Subject)\nRETURN s.title AS ...
4,Apa pasal selanjutnya setelah Pasal 39 UU Nomo...,[{'text': 'Undang-undang (UU) Nomor 11 Tahun 2...,"[{'next_article': {'number': '40', 'chapter': ...",Pasal selanjutnya setelah Pasal 39 UU Nomor 11...,1.0,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
5,Apa pasal sebelum Pasal 41 UU Nomor 11 Tahun 2...,[{'text': 'Undang-undang (UU) Nomor 11 Tahun 2...,"[{'previous_article': {'number': '40', 'chapte...",Pasal sebelum Pasal 41 UU Nomor 11 Tahun 2008 ...,1.0,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
6,Apakah Pasal 40 UU Nomor 11 Tahun 2008 pernah ...,"[{'amended_article': {'number': '40', 'chapter...","[{'amend_article': {'number': '40', 'chapter':...","Ya, Pasal 40 UU Nomor 11 Tahun 2008 pernah dia...",1.0,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...


In [ ]:
# PAKAI QA LLM
df_gpt_eval_result = results.to_pandas()
df_gpt_eval_result["generated_cypher"] = generated_cypher_results
df_gpt_eval_result

,user_input,retrieved_contexts,reference,context_recall,llm_context_precision_with_reference,kesesuaian_tipe_1,kesesuaian_tipe_2,generated_cypher
0,Apa judul dari Peraturan Pemerintah (PP) Nomor...,[Judul dari Peraturan Pemerintah (PP) Nomor 80...,Judul dari Peraturan Pemerintah (PP) Nomor 80 ...,1.00,1.0,1,90,cypher \nMATCH (r:Regulation)\nWHERE r.type = ...
1,Apa isi Pasal 2 Peraturan Pemerintah (PP) Nomo...,[Sesuai dengan Pasal 2 Peraturan Pemerintah (P...,Isi Pasal 2 Peraturan Pemerintah (PP) Nomor 80...,1.00,1.0,1,95,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
2,Apa isi Pasal 10 Peraturan Menteri Komunikasi ...,[Sesuai dengan Pasal 10 Peraturan Menteri Komu...,Isi Pasal 10 Peraturan Menteri Komunikasi dan ...,1.00,1.0,1,95,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
3,Berdasarkan daftar peraturan teknologi informa...,[Daftar peraturan teknologi informasi yang ada...,Berikut adalah daftar topik atau subjek peratu...,1.00,1.0,1,90,cypher \nMATCH (s:Subject)\nRETURN s.title AS ...
4,Apa judul pasal selanjutnya setelah Pasal 39 U...,[Judul pasal selanjutnya setelah Pasal 39 UU N...,Pasal selanjutnya setelah Pasal 39 UU Nomor 11...,0.25,1.0,1,85,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
5,Apa judul pasal sebelum Pasal 41 UU Nomor 11 T...,[Judul pasal sebelum Pasal 41 UU Nomor 11 Tahu...,Pasal sebelum Pasal 41 UU Nomor 11 Tahun 2008 ...,0.00,1.0,0,60,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
6,Apakah Pasal 40 UU Nomor 11 Tahun 2008 pernah ...,"[Ya, Pasal 40 UU Nomor 11 Tahun 2008 tentang I...","Ya, Pasal 40 UU Nomor 11 Tahun 2008 pernah dia...",0.80,1.0,1,90,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...


In [37]:
# ISI CONTEXT DI ITERASI KE DALAM LIST (SKIP QA LLM)
df_gpt_eval_result = results.to_pandas()
df_gpt_eval_result["generated_cypher"] = generated_cypher_results
df_gpt_eval_result

,user_input,retrieved_contexts,reference_contexts,reference,non_llm_context_recall,non_llm_context_precision_with_reference,generated_cypher
0,Apa judul dari Peraturan Pemerintah (PP) Nomor...,[{'title': 'Peraturan Pemerintah (PP) Nomor 80...,[Peraturan Pemerintah (PP) Nomor 80 Tahun 2019...,Judul dari Peraturan Pemerintah (PP) Nomor 80 ...,1.0,1.0,cypher \nMATCH (r:Regulation)\nWHERE r.type = ...
1,Apa isi Pasal 2 Peraturan Pemerintah (PP) Nomo...,[{'text': 'Peraturan Pemerintah (PP) Nomor 80 ...,[Peraturan Pemerintah (PP) Nomor 80 Tahun 2019...,Isi Pasal 2 Peraturan Pemerintah (PP) Nomor 80...,1.0,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
2,Apa isi Pasal 10 Peraturan Menteri Komunikasi ...,[{'text': 'Peraturan Menteri Komunikasi dan In...,[Peraturan Menteri Komunikasi dan Informatika ...,Isi Pasal 10 Peraturan Menteri Komunikasi dan ...,1.0,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
3,Berdasarkan daftar peraturan teknologi informa...,"[{'Subject': 'TELEKOMUNIKASI, INFORMATIKA, SIB...",[{'subject': 'ADMINISTRASI DAN TATA USAHA NEGA...,Berikut adalah daftar topik atau subjek peratu...,1.0,1.0,cypher \nMATCH (r:Regulation)-[:HAS_SUBJECT]->...
4,Apa pasal selanjutnya setelah Pasal 39 UU Nomo...,[{'text': 'Undang-undang (UU) Nomor 11 Tahun 2...,"[{'next_article': {'number': '40', 'chapter': ...",Pasal selanjutnya setelah Pasal 39 UU Nomor 11...,1.0,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
5,Apa pasal sebelum Pasal 41 UU Nomor 11 Tahun 2...,[{'text': 'Undang-undang (UU) Nomor 11 Tahun 2...,"[{'previous_article': {'number': '40', 'chapte...",Pasal sebelum Pasal 41 UU Nomor 11 Tahun 2008 ...,1.0,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
6,Apakah Pasal 40 UU Nomor 11 Tahun 2008 pernah ...,[{'original_article_text': 'Undang-undang (UU)...,"[{'amend_article': {'number': '40', 'chapter':...","Ya, Pasal 40 UU Nomor 11 Tahun 2008 pernah dia...",0.5,0.5,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
7,Apa saja pertimbangan utama atau latar belakan...,[{'consideration_text': 'Daftar Latar Belakang...,"[{'target': {'id': 201603010200000, 'text': 'D...",Pertimbangan utama yang mendasari pembuatan PE...,1.0,1.0,cypher \nMATCH (r:Regulation)-[:HAS_CONSIDERAT...
8,Apa saja dasar hukum yang mendasari pembuatan ...,[{'Dasar_Hukum': 'Daftar Dasar Hukum dari Pera...,"[{'target': {'id': 201603010300000, 'text': 'D...","Berdasarkan hasil kueri, dasar hukum yang mend...",1.0,1.0,cypher \nMATCH (r:Regulation)-[:HAS_OBSERVATIO...
9,Apa saja pasal yang dirujuk oleh Pasal 25 PERM...,[{'text': 'Peraturan Menteri Komunikasi dan In...,"[{'refered_article': {'number': '10', 'chapter...",Pasal 25 PERMENKOMINFO No. 4 Tahun 2016 meruju...,1.0,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...


In [13]:
# ISI CONTEXT DI ITERASI KE DALAM LIST (SKIP QA LLM)
df_gpt_eval_result = results.to_pandas()
df_gpt_eval_result["generated_cypher"] = generated_cypher_results
df_gpt_eval_result

,user_input,retrieved_contexts,reference_contexts,reference,context_recall,non_llm_context_recall,llm_context_precision_with_reference,non_llm_context_precision_with_reference,generated_cypher
0,Apa judul dari Peraturan Pemerintah (PP) Nomor...,[{'title': 'Peraturan Pemerintah (PP) Nomor 80...,[Peraturan Pemerintah (PP) Nomor 80 Tahun 2019...,Judul dari Peraturan Pemerintah (PP) Nomor 80 ...,1.0,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)\nWHERE r.type = ...
1,Apa isi Pasal 2 Peraturan Pemerintah (PP) Nomo...,[{'text': 'Peraturan Pemerintah (PP) Nomor 80 ...,[Peraturan Pemerintah (PP) Nomor 80 Tahun 2019...,Isi Pasal 2 Peraturan Pemerintah (PP) Nomor 80...,1.0,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
2,Apa isi Pasal 10 Peraturan Menteri Komunikasi ...,[{'text': 'Peraturan Menteri Komunikasi dan In...,[Peraturan Menteri Komunikasi dan Informatika ...,Isi Pasal 10 Peraturan Menteri Komunikasi dan ...,1.0,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
3,Berdasarkan daftar peraturan teknologi informa...,"[{'title': 'TELEKOMUNIKASI, INFORMATIKA, SIBER...",[{'title': 'ADMINISTRASI DAN TATA USAHA NEGARA...,Berikut adalah daftar topik atau subjek peratu...,1.0,1.0,0.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_SUBJECT]->...
4,Apa judul pasal selanjutnya setelah Pasal 39 U...,"[{'title': 'Pasal 40 UU Nomor 11 Tahun 2008'},...","[{'title': 'Pasal 40 UU Nomor 11 Tahun 2008', ...",Pasal selanjutnya setelah Pasal 39 UU Nomor 11...,1.0,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
5,Apa pasal sebelum Pasal 41 UU Nomor 11 Tahun 2...,[{'text': 'Undang-undang (UU) Nomor 11 Tahun 2...,"[{'title': 'Pasal 40 UU Nomor 11 Tahun 2008', ...",Pasal sebelum Pasal 41 UU Nomor 11 Tahun 2008 ...,1.0,0.0,0.583333,0.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
6,Apakah Pasal 40 UU Nomor 11 Tahun 2008 pernah ...,[{'title': 'Pasal 40 UU Nomor 19 Tahun 2016 te...,[{'title': 'Pasal 40 UU Nomor 19 Tahun 2016 te...,"Ya, Pasal 40 UU Nomor 11 Tahun 2008 pernah dia...",1.0,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
7,Apa saja pertimbangan utama atau latar belakan...,[{'text': 'Daftar Latar Belakang dari Peratura...,[{'text': 'Daftar Latar Belakang dari Peratura...,Pertimbangan utama yang mendasari pembuatan PE...,1.0,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_CONSIDERAT...
8,Apa saja dasar hukum yang mendasari pembuatan ...,[{'text': 'Daftar Dasar Hukum dari Peraturan M...,['text': 'Daftar Dasar Hukum dari Peraturan Me...,"Berdasarkan hasil kueri, dasar hukum yang mend...",1.0,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_OBSERVATIO...
9,Apa saja judul pasal yang dirujuk oleh Pasal 2...,[{'title': 'Pasal 10 PERMENKOMINFO Nomor 4 Tah...,[{'title': 'Pasal 10 PERMENKOMINFO Nomor 4 Tah...,Pasal 25 PERMENKOMINFO No. 4 Tahun 2016 meruju...,1.0,1.0,0.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...


In [18]:
# ISI CONTEXT DI ITERASI KE DALAM LIST (SKIP QA LLM)
df_gpt_eval_result = results.to_pandas()
df_gpt_eval_result["generated_cypher"] = generated_cypher_results[3:7]
df_gpt_eval_result

,user_input,retrieved_contexts,reference_contexts,reference,context_recall,non_llm_context_recall,llm_context_precision_with_reference,non_llm_context_precision_with_reference,generated_cypher
0,Berdasarkan daftar peraturan teknologi informa...,"[{'title': 'TELEKOMUNIKASI, INFORMATIKA, SIBER...",[{'subject': 'ADMINISTRASI DAN TATA USAHA NEGA...,Berikut adalah daftar topik atau subjek peratu...,1.0,1.0,0.277778,1.0,cypher \nMATCH (r:Regulation)-[:HAS_SUBJECT]->...
1,Apa judul pasal selanjutnya setelah Pasal 39 U...,"[{'title': 'Pasal 40 UU Nomor 11 Tahun 2008'},...","[{'title': 'Pasal 40 UU Nomor 11 Tahun 2008', ...",Pasal selanjutnya setelah Pasal 39 UU Nomor 11...,1.0,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
2,Apa pasal sebelum Pasal 41 UU Nomor 11 Tahun 2...,[{'text': 'Undang-undang (UU) Nomor 11 Tahun 2...,"[{'title': 'Pasal 40 UU Nomor 11 Tahun 2008', ...",Pasal sebelum Pasal 41 UU Nomor 11 Tahun 2008 ...,1.0,0.0,0.833333,0.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
3,Apakah Pasal 40 UU Nomor 11 Tahun 2008 pernah ...,[{'title': 'Pasal 40 UU Nomor 19 Tahun 2016 te...,[{'title': 'Pasal 40 UU Nomor 19 Tahun 2016 te...,"Ya, Pasal 40 UU Nomor 11 Tahun 2008 pernah dia...",1.0,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...


In [39]:
# ISI CONTEXT DI ITERASI KE DALAM LIST (SKIP QA LLM)
df_gpt_eval_result = results.to_pandas()
df_gpt_eval_result["generated_cypher"] = generated_cypher_results
df_gpt_eval_result

,user_input,retrieved_contexts,reference_contexts,reference,context_recall,non_llm_context_recall,llm_context_precision_with_reference,non_llm_context_precision_with_reference,generated_cypher
0,Apa judul dari Peraturan Pemerintah (PP) Nomor...,[{'title': 'Peraturan Pemerintah (PP) Nomor 80...,[Peraturan Pemerintah (PP) Nomor 80 Tahun 2019...,Judul dari Peraturan Pemerintah (PP) Nomor 80 ...,1.000000,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)\nWHERE r.type = ...
1,Apa isi Pasal 2 Peraturan Pemerintah (PP) Nomo...,[{'text': 'Peraturan Pemerintah (PP) Nomor 80 ...,[Peraturan Pemerintah (PP) Nomor 80 Tahun 2019...,Isi Pasal 2 Peraturan Pemerintah (PP) Nomor 80...,1.000000,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
2,Apa isi Pasal 10 Peraturan Menteri Komunikasi ...,[{'text': 'Peraturan Menteri Komunikasi dan In...,[Peraturan Menteri Komunikasi dan Informatika ...,Isi Pasal 10 Peraturan Menteri Komunikasi dan ...,1.000000,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
3,Berdasarkan daftar peraturan teknologi informa...,"[{'Subject': 'TELEKOMUNIKASI, INFORMATIKA, SIB...",[{'subject': 'ADMINISTRASI DAN TATA USAHA NEGA...,Berikut adalah daftar topik atau subjek peratu...,1.000000,1.0,0.237716,1.0,cypher \nMATCH (r:Regulation)-[:HAS_SUBJECT]->...
4,Apa pasal selanjutnya setelah Pasal 39 UU Nomo...,[{'text': 'Undang-undang (UU) Nomor 11 Tahun 2...,"[{'next_article': {'number': '40', 'chapter': ...",Pasal selanjutnya setelah Pasal 39 UU Nomor 11...,1.000000,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
5,Apa pasal sebelum Pasal 41 UU Nomor 11 Tahun 2...,[{'text': 'Undang-undang (UU) Nomor 11 Tahun 2...,"[{'previous_article': {'number': '40', 'chapte...",Pasal sebelum Pasal 41 UU Nomor 11 Tahun 2008 ...,1.000000,1.0,0.583333,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
6,Apakah Pasal 40 UU Nomor 11 Tahun 2008 pernah ...,[{'original_article_text': 'Undang-undang (UU)...,"[{'amend_article': {'number': '40', 'chapter':...","Ya, Pasal 40 UU Nomor 11 Tahun 2008 pernah dia...",1.000000,0.5,1.000000,0.5,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
7,Apa saja pertimbangan utama atau latar belakan...,[{'consideration_text': 'Daftar Latar Belakang...,"[{'target': {'id': 201603010200000, 'text': 'D...",Pertimbangan utama yang mendasari pembuatan PE...,1.000000,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_CONSIDERAT...
8,Apa saja dasar hukum yang mendasari pembuatan ...,[{'Dasar_Hukum': 'Daftar Dasar Hukum dari Pera...,"[{'target': {'id': 201603010300000, 'text': 'D...","Berdasarkan hasil kueri, dasar hukum yang mend...",1.000000,1.0,1.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_OBSERVATIO...
9,Apa saja pasal yang dirujuk oleh Pasal 25 PERM...,[{'text': 'Peraturan Menteri Komunikasi dan In...,"[{'refered_article': {'number': '10', 'chapter...",Pasal 25 PERMENKOMINFO No. 4 Tahun 2016 meruju...,0.666667,1.0,0.000000,1.0,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...


In [ ]:
df_gemini_eval_result = results.to_pandas()
df_gemini_eval_result["generated_cypher"] = generated_cypher_results
df_gemini_eval_result.to_excel("text2cypher_evaluation_result.xlsx")
df_gemini_eval_result

,user_input,retrieved_contexts,reference,kesesuaian_tipe_1,kesesuaian_tipe_2,generated_cypher
0,Apa judul dari Peraturan Pemerintah (PP) Nomor...,[{'title': 'Peraturan Pemerintah (PP) Nomor 80...,Judul dari Peraturan Pemerintah (PP) Nomor 80 ...,1,100,cypher \nMATCH (r:Regulation)\nWHERE r.type = ...
1,Apa isi Pasal 2 Peraturan Pemerintah (PP) Nomo...,[{'text': 'Peraturan Pemerintah (PP) Nomor 80 ...,Isi Pasal 2 Peraturan Pemerintah (PP) Nomor 80...,1,100,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
2,Apa isi Pasal 10 Peraturan Menteri Komunikasi ...,[{'text': 'Peraturan Menteri Komunikasi dan In...,Isi Pasal 10 Peraturan Menteri Komunikasi dan ...,1,100,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
3,Berdasarkan daftar peraturan teknologi informa...,"[{'Subject': 'TELEKOMUNIKASI, INFORMATIKA, SIB...",Berikut adalah daftar topik atau subjek peratu...,1,100,cypher \nMATCH (s:Subject)\nRETURN s.title AS ...
4,Apa judul pasal selanjutnya setelah Pasal 39 U...,"[{'title': 'Pasal 40 UU Nomor 11 Tahun 2008'},...",Pasal selanjutnya setelah Pasal 39 UU Nomor 11...,1,95,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
5,Apa judul pasal sebelum Pasal 41 UU Nomor 11 T...,"[{'title': 'Pasal 40 UU Nomor 11 Tahun 2008'},...",Pasal sebelum Pasal 41 UU Nomor 11 Tahun 2008 ...,1,75,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
6,Apakah Pasal 40 UU Nomor 11 Tahun 2008 pernah ...,"[{'amended_article': {'number': '40', 'chapter...","Ya, Pasal 40 UU Nomor 11 Tahun 2008 pernah dia...",1,100,cypher \nMATCH (r:Regulation)-[:HAS_ARTICLE]->...
